In [ ]:
from otoole import ReadCsv
import os

import xarray as xr
from numpy import inf

from otoole.utils import (
    _read_file,
    validate_config
)

import logging

from variables import add_variables

logger = logging.getLogger(__name__)

config_path = "config.yaml"
folder_path = os.path.join('test', 'simplicity')
# folder_path = os.path.join('test', 'super_simple')

with open(config_path, "r") as config_file:
    config = _read_file(config_file, '.yaml')
validate_config(config)

read_strategy = ReadCsv(user_config=config)

model, defaults = read_strategy.read(folder_path)
logging.debug(model.keys())

In [ ]:
data_vars = {x: y.VALUE.to_xarray() for x, y in model.items() if config[x]['type'] == 'param'}
coords = {x: y.values.T[0] for x, y in model.items() if config[x]['type'] == 'set'}
ds = xr.Dataset(data_vars=data_vars, coords=coords)
ds = ds.assign_coords({'_REGION': model['REGION'].values.T[0]})

In [ ]:
for param, default in defaults.items():
    if default != 0:
        ds[param] = ds[param].fillna(default)

# Model Creation

In [ ]:
from linopy import Model, solvers, available_solvers
m = Model(force_dim_names=True)

## Variables

In [ ]:
m = add_variables(ds, m)

In [ ]:
RRTiFY = [ds.coords['REGION'], ds.coords['_REGION'], ds.coords['TIMESLICE'], ds.coords['FUEL'], ds.coords['YEAR']]
RRFY = [ds.coords['REGION'], ds.coords['_REGION'], ds.coords['FUEL'], ds.coords['YEAR']]
RFY = [ds.coords['REGION'], ds.coords['FUEL'], ds.coords['YEAR']]
RTiFY = [ds.coords['REGION'], ds.coords['TIMESLICE'], ds.coords['FUEL'], ds.coords['YEAR']]
RSY = [ds.coords['REGION'], ds.coords['STORAGE'], ds.coords['YEAR']]
RTeY = [ds.coords['REGION'], ds.coords['TECHNOLOGY'], ds.coords['YEAR']]

# Discounting

```ampl
param DiscountRate{r in REGION};
param DiscountRateIdv{r in REGION, t in TECHNOLOGY}, default DiscountRate[r];

param DiscountFactor{r in REGION, y in YEAR} :=
	(1 + DiscountRate[r]) ^ (y - min{yy in YEAR} min(yy) + 0.0);
param DiscountFactorMid{r in REGION, y in YEAR} :=
	(1 + DiscountRate[r]) ^ (y - min{yy in YEAR} min(yy) + 0.5);

param OperationalLife{r in REGION, t in TECHNOLOGY};

param CapitalRecoveryFactor{r in REGION, t in TECHNOLOGY} :=
	(1 - (1 + DiscountRateIdv[r,t])^(-1))/(1 - (1 + DiscountRateIdv[r,t])^(-(OperationalLife[r,t])));
param PvAnnuity{r in REGION, t in TECHNOLOGY} :=
	(1 - (1 + DiscountRate[r])^(-(OperationalLife[r,t]))) * (1 + DiscountRate[r]) / DiscountRate[r];

param DiscountRateStorage{r in REGION, s in STORAGE};
param DiscountFactorStorage{r in REGION, s in STORAGE, y in YEAR} :=
	(1 + DiscountRateStorage[r, s]) ^ (y - min{yy in YEAR} min(yy) + 0.0);
param DiscountFactorMidStorage{r in REGION, s in STORAGE, y in YEAR} :=
	(1 + DiscountRateStorage[r, s]) ^ (y - min{yy in YEAR} min(yy) + 0.5);
```

In [ ]:
discount_factor = ((1 + ds['DiscountRate']) ** -(ds.coords['YEAR'] - min(ds.coords['YEAR'])))
discount_factor_mid = ((1 + ds['DiscountRate']) ** -(ds.coords['YEAR'] - min(ds.coords['YEAR']) + 0.5))

In [ ]:
discount_factor.plot()

In [ ]:
discount_factor_idv = ((1 + ds['DiscountRateIdv']) ** -(ds.coords['YEAR'] - min(ds.coords['YEAR'])))
discount_factor_mid_idv = ((1 + ds['DiscountRateIdv']) ** -(ds.coords['YEAR'] - min(ds.coords['YEAR']) + 0.5))

In [ ]:
pv_annuity = (1 - (1 + ds['DiscountRateIdv'])**(-(ds['OperationalLife']))) * (1 + ds['DiscountRateIdv']) / ds['DiscountRateIdv']
pv_annuity

In [ ]:
capital_recovery_factor = (1 - (1 + ds['DiscountRateIdv'])**(-1))/(1 - (1 + ds['DiscountRateIdv'])**(-(ds['OperationalLife'])))
capital_recovery_factor

In [ ]:
capital_recovery_factor.to_dataframe(name='crf').plot(kind='bar')

In [ ]:
pv_annuity * capital_recovery_factor

# Constraints

## Demand

```ampl
s.t. EQ_SpecifiedDemand{r in REGION, l in TIMESLICE, f in FUEL, y in YEAR:
						SpecifiedAnnualDemand[r,f,y] <> 0}:
	SpecifiedAnnualDemand[r,f,y] * SpecifiedDemandProfile[r,f,l,y] / YearSplit[l,y]
	=
	RateOfDemand[r,l,f,y];
	```

In [ ]:
mask = ~ds['SpecifiedAnnualDemand'].isnull()
con = m['RateOfDemand'] == (ds['SpecifiedAnnualDemand'] * ds['SpecifiedDemandProfile'] / ds['YearSplit'])
m.add_constraints(con, name='EQ_SpecifiedDemand', mask=mask)

## Capacity Adequacy A

```ampl
s.t. CAa1_TotalNewCapacity{r in REGION, t in TECHNOLOGY, y in YEAR}:
	AccumulatedNewCapacity[r,t,y]
	=
	sum{yy in YEAR: y-yy < OperationalLife[r,t] && y - yy >= 0} NewCapacity[r,t,yy];
```

In [ ]:
def bounds(model, r, t, y):
    return m['AccumulatedNewCapacity'][r,t,y] - 1 * \
                sum(m['NewCapacity'][r,t,yy] for yy in ds.coords['YEAR'].values 
                    if (y-yy >= 0) and (y-yy < ds['OperationalLife'].sel({'REGION':r, 'TECHNOLOGY': t}))) == 0
    
m.add_constraints(bounds, coords=RTeY, name='CAa1_TotalNewCapacity')

```ampl
s.t. CAa2_TotalAnnualCapacity{r in REGION, t in TECHNOLOGY, y in YEAR}:
	AccumulatedNewCapacity[r,t,y] + ResidualCapacity[r,t,y]
	=
	TotalCapacityAnnual[r,t,y];
```

In [ ]:
con = m['AccumulatedNewCapacity'] - m['TotalCapacityAnnual'] == -ds['ResidualCapacity'].fillna(0)
m.add_constraints(con, name="CAa2_TotalAnnualCapacity", coords=RTeY)

```ampl
s.t. CAa3_TotalActivityOfEachTechnology{r in REGION, t in TECHNOLOGY, l in TIMESLICE, y in YEAR}:
	sum{m in MODE_OF_OPERATION} RateOfActivity[r,l,t,m,y]
	=
	RateOfTotalActivity[r,t,l,y];
```

In [ ]:
con = m['RateOfActivity'].sum(dims='MODE_OF_OPERATION') - m['RateOfTotalActivity'] == 0
m.add_constraints(con, name='CAa3_TotalActivityOfEachTechnology')

```ampl
s.t. CAa4_Constraint_Capacity{r in REGION, l in TIMESLICE, t in TECHNOLOGY, y in YEAR}:
	RateOfTotalActivity[r,t,l,y]
	<=
	TotalCapacityAnnual[r,t,y] * CapacityFactor[r,t,l,y] * CapacityToActivityUnit[r,t];
```

In [ ]:
con = m['RateOfTotalActivity'] - (m['TotalCapacityAnnual'] * ds['CapacityFactor'] * ds['CapacityToActivityUnit']) <=0
mask = ~ds['CapacityFactor'].isnull()
m.add_constraints(con, name='CAa4_Constraint_Capacity', mask=mask)

```ampl
s.t. CAa5_TotalNewCapacity{r in REGION, t in TECHNOLOGY, y in YEAR: CapacityOfOneTechnologyUnit[r,t,y]<>0}:
	CapacityOfOneTechnologyUnit[r,t,y] * NumberOfNewTechnologyUnits[r,t,y]
	=
	NewCapacity[r,t,y];
```

In [ ]:
con = ds['CapacityOfOneTechnologyUnit'] * m['NumberOfNewTechnologyUnits'] - m['NewCapacity'] == 0
mask = ~ds['CapacityOfOneTechnologyUnit'].isnull()
m.add_constraints(con, name='CAa5_TotalNewCapacity', mask=mask)

## Capacity Adequacy B
```ampl
s.t. CAb1_PlannedMaintenance{r in REGION, t in TECHNOLOGY, y in YEAR: AvailabilityFactor[r,t,y] < 1}:
	sum{l in TIMESLICE} RateOfTotalActivity[r,t,l,y] * YearSplit[l,y]
	<=
	sum{l in TIMESLICE} (TotalCapacityAnnual[r,t,y] * CapacityFactor[r,t,l,y] * YearSplit[l,y])
	* AvailabilityFactor[r,t,y] * CapacityToActivityUnit[r,t];
```

In [ ]:
mask = ds['AvailabilityFactor'] < 1
con = (m['RateOfTotalActivity'] * ds['YearSplit']).sum(dims='TIMESLICE') \
      - (m['TotalCapacityAnnual'] * ds['CapacityFactor'] * ds['YearSplit']).sum(dims='TIMESLICE') * ds['AvailabilityFactor'] * ds['CapacityToActivityUnit'] <= 0
m.add_constraints(con, name='CAb1_PlannedMaintenance', coords=RTeY, mask=mask)

## Energy Balance A

```ampl
s.t. EBa1_RateOfFuelProduction1{
	r in REGION, l in TIMESLICE, f in FUEL, t in TECHNOLOGY, m in MODE_OF_OPERATION, y in YEAR:
	OutputActivityRatio[r,t,f,m,y] <> 0}:
	RateOfActivity[r,l,t,m,y] * OutputActivityRatio[r,t,f,m,y]
	=
	RateOfProductionByTechnologyByMode[r,l,t,m,f,y];
```

In [ ]:
mask = ~ds['OutputActivityRatio'].isnull()
con = m['RateOfActivity'] * ds['OutputActivityRatio'] - m['RateOfProductionByTechnologyByMode'] == 0
m.add_constraints(con, name='EBa1_RateOfFuelProduction1', mask=mask)

```ampl
s.t. EBa2_RateOfFuelProduction2{r in REGION, l in TIMESLICE, f in FUEL, t in TECHNOLOGY, y in YEAR}:
	sum{m in MODE_OF_OPERATION: OutputActivityRatio[r,t,f,m,y] <> 0} RateOfProductionByTechnologyByMode[r,l,t,m,f,y]
	=
	RateOfProductionByTechnology[r,l,t,f,y];
```

In [ ]:
mask = ds['OutputActivityRatio'].sum(dim='MODE_OF_OPERATION') != 0
con = (m['RateOfProductionByTechnologyByMode']).sum(dims='MODE_OF_OPERATION') - m['RateOfProductionByTechnology'] == 0
m.add_constraints(con, name='EBa2_RateOfFuelProduction2', mask=mask)

```ampl
s.t. EBa3_RateOfFuelProduction3{r in REGION, l in TIMESLICE, f in FUEL, y in YEAR:
							    (sum{t in TECHNOLOGY, m in MODE_OF_OPERATION} OutputActivityRatio[r,t,f,m,y]) <> 0}:
	sum{t in TECHNOLOGY} RateOfProductionByTechnology[r,l,t,f,y]
	=
	RateOfProduction[r,l,f,y];
```

In [ ]:
con = m['RateOfProductionByTechnology'].sum(dims='TECHNOLOGY') - m['RateOfProduction'] == 0
mask = ~ds['OutputActivityRatio'].sum(dim=['TECHNOLOGY', 'MODE_OF_OPERATION']).isnull()
m.add_constraints(con, name='EBa3_RateOfFuelProduction3', mask=mask)

```ampl
s.t. EBa4_RateOfFuelUse1{r in REGION, l in TIMESLICE, f in FUEL, t in TECHNOLOGY, m in MODE_OF_OPERATION, y in YEAR:
						 InputActivityRatio[r,t,f,m,y] <> 0}:
	RateOfActivity[r,l,t,m,y] * InputActivityRatio[r,t,f,m,y]
	=
	RateOfUseByTechnologyByMode[r,l,t,m,f,y];
```

In [ ]:
con = m['RateOfActivity'] * ds['InputActivityRatio'] - m['RateOfUseByTechnologyByMode'] == 0
mask = ~ds['InputActivityRatio'].isnull()
m.add_constraints(con, name='EBa4_RateOfFuelUse1', mask=mask)

```ampl
s.t. EBa5_RateOfFuelUse2{r in REGION, l in TIMESLICE, f in FUEL, t in TECHNOLOGY, y in YEAR:
						 sum{m in MODE_OF_OPERATION} InputActivityRatio[r,t,f,m,y] <> 0}:
	sum{m in MODE_OF_OPERATION: InputActivityRatio[r,t,f,m,y] <> 0}
	RateOfUseByTechnologyByMode[r,l,t,m,f,y]
	=
	RateOfUseByTechnology[r,l,t,f,y];
```

In [ ]:
con = m['RateOfUseByTechnologyByMode'].sum(dims='MODE_OF_OPERATION') - m['RateOfUseByTechnology'] == 0
mask = ds['InputActivityRatio'].sum(dim='MODE_OF_OPERATION') != 0
m.add_constraints(con, name='EBa5_RateOfFuelUse2', mask=mask)

```ampl
s.t. EBa6_RateOfFuelUse3{r in REGION, l in TIMESLICE, f in FUEL, y in YEAR:
						 sum{t in TECHNOLOGY, m in MODE_OF_OPERATION} InputActivityRatio[r,t,f,m,y] <> 0}:
	sum{t in TECHNOLOGY} RateOfUseByTechnology[r,l,t,f,y]
	=
	RateOfUse[r,l,f,y];
```

In [ ]:
con = m['RateOfUseByTechnology'].sum(dims='TECHNOLOGY') - m['RateOfUse'] == 0
mask = ds['InputActivityRatio'].sum(dim=['TECHNOLOGY', 'MODE_OF_OPERATION']) != 0
m.add_constraints(con, name='EBa6_RateOfFuelUse3', mask=mask)

```ampl
s.t. EBa7_EnergyBalanceEachTS1{r in REGION, l in TIMESLICE, f in FUEL, y in YEAR:
							   (sum{t in TECHNOLOGY, m in MODE_OF_OPERATION} OutputActivityRatio[r,t,f,m,y]) <> 0}:
	RateOfProduction[r,l,f,y] * YearSplit[l,y]
	=
	Production[r,l,f,y];
```

In [ ]:
con = m['RateOfProduction'] * ds['YearSplit'] - m['Production'] == 0
mask = ds['OutputActivityRatio'].sum(dim=['TECHNOLOGY', 'MODE_OF_OPERATION']) != 0
m.add_constraints(con, name='EBa7_EnergyBalanceEachTS1', mask=mask)

```ampl
s.t. EBa8_EnergyBalanceEachTS2{r in REGION, l in TIMESLICE, f in FUEL, y in YEAR:
							   (sum{t in TECHNOLOGY, m in MODE_OF_OPERATION} InputActivityRatio[r,t,f,m,y]) <> 0}:
	RateOfUse[r,l,f,y] * YearSplit[l,y]
	=
	Use[r,l,f,y];
```

In [ ]:
con = (m['RateOfUse'] * ds['YearSplit']) - m['Use'] == 0
mask = ds['InputActivityRatio'].sum(dim=['TECHNOLOGY', 'MODE_OF_OPERATION']) != 0
m.add_constraints(con, name='EBa8_EnergyBalanceEachTS2', mask=mask)

```ampl
s.t. EBa9_EnergyBalanceEachTS3{r in REGION, l in TIMESLICE, f in FUEL, y in YEAR:
							   SpecifiedAnnualDemand[r,f,y] <> 0}:
	RateOfDemand[r,l,f,y] * YearSplit[l,y]
	=
	Demand[r,l,f,y];
```

In [ ]:
con = m['RateOfDemand'] * ds['YearSplit'] - m['Demand'] == 0
mask = ~ds['SpecifiedAnnualDemand'].isnull()
m.add_constraints(con, name='EBa9_EnergyBalanceEachTS3', mask=mask)

```ampl
s.t. EBa10_EnergyBalanceEachTS4{r in REGION, rr in REGION, l in TIMESLICE, f in FUEL, y in YEAR:
								TradeRoute[r,rr,f,y] <> 0}:
	Trade[r,rr,l,f,y]
	=
	-Trade[rr,r,l,f,y];
```

```python
def EnergyBalanceEachTS4_rule(model, r, rr, l, f, y):
    return model.Trade[r, rr, l, f, y] + model.Trade[rr, r, l, f, y] == 0
```

In [ ]:
def energy_balance_each_ts4_rule(model, r, rr, l, f, y):
    return m['Trade'][r,rr,l,f,y] + m['Trade'][rr,r,l,f,y] == 0
tr = ds['TradeRoute']
mask = ~tr.where(tr.REGION != tr._REGION).isnull()
m.add_constraints(energy_balance_each_ts4_rule, name='EBa10_EnergyBalanceEachTS4', mask=mask, coords=RRTiFY)

```ampl
s.t. EBa11_EnergyBalanceEachTS5{r in REGION, l in TIMESLICE, f in FUEL, y in YEAR}:
	Production[r,l,f,y]
	>=
	Demand[r,l,f,y] + Use[r,l,f,y] + sum{rr in REGION} Trade[r,rr,l,f,y] * TradeRoute[r,rr,f,y];
```

In [ ]:
con = m['Production'] - (m['Demand'] + m['Use'] + (m['Trade'] * ds['TradeRoute'].fillna(0)).sum(dims='_REGION')) >= 0
m.add_constraints(con, name='EBa11_EnergyBalanceEachTS5')

# Energy Balance B

```ampl
s.t. EBb1_EnergyBalanceEachYear1{r in REGION, f in FUEL, y in YEAR}:
	sum{l in TIMESLICE} Production[r,l,f,y]
	=
	ProductionAnnual[r,f,y];
```

In [ ]:
con = m['Production'].sum(dims='TIMESLICE') - m['ProductionAnnual'] == 0
m.add_constraints(con, name='EBb1_EnergyBalanceEachYear1')

```ampl
s.t. EBb2_EnergyBalanceEachYear2{r in REGION, f in FUEL, y in YEAR}:
	sum{l in TIMESLICE} Use[r,l,f,y]
	=
	UseAnnual[r,f,y];
```

In [ ]:
con = m['Use'].sum('TIMESLICE') - m['UseAnnual'] == 0
ebb2_energy_balance_each_year2 = m.add_constraints(con, name='EBb2_EnergyBalanceEachYear2')

In [ ]:
ebb2_energy_balance_each_year2

```ampl
s.t. EBb3_EnergyBalanceEachYear3{r in REGION, rr in REGION, f in FUEL, y in YEAR}:
	sum{l in TIMESLICE} Trade[r,rr,l,f,y]
	=
	TradeAnnual[r,rr,f,y];
```

In [ ]:
con = m['Trade'].sum('TIMESLICE') - m['TradeAnnual'] == 0
mask = ds.coords['REGION'] != ds.coords['_REGION']
m.add_constraints(con, name='EBb3_EnergyBalanceEachYear3', mask=mask)

```ampl
s.t. EBb4_EnergyBalanceEachYear4{r in REGION, f in FUEL, y in YEAR}:
	ProductionAnnual[r,f,y]
	>=
	UseAnnual[r,f,y] + sum{rr in REGION} TradeAnnual[r,rr,f,y] * TradeRoute[r,rr,f,y] + AccumulatedAnnualDemand[r,f,y];
```

In [ ]:
con = m['ProductionAnnual'] - m['UseAnnual'] - (m['TradeAnnual'].sum('_REGION') * ds['TradeRoute'].sum('_REGION')) >= ds['AccumulatedAnnualDemand'].fillna(0)
m.add_constraints(con, name='EBb4_EnergyBalanceEachYear4')

# Accounting Technology Production/Use

```ampl
s.t. Acc1_FuelProductionByTechnology{r in REGION, l in TIMESLICE, t in TECHNOLOGY, f in FUEL, y in YEAR}:
	RateOfProductionByTechnology[r,l,t,f,y] * YearSplit[l,y]
	=
	ProductionByTechnology[r,l,t,f,y];
```

In [ ]:
con = (m['RateOfProductionByTechnology'] * ds['YearSplit']) - m['ProductionByTechnology'] == 0
mask = ds['OutputActivityRatio'].sum('MODE_OF_OPERATION') != 0
m.add_constraints(con, name='Acc1_FuelProductionByTechnology', mask=mask)

```ampl
s.t. Acc2_FuelUseByTechnology{r in REGION, l in TIMESLICE, t in TECHNOLOGY, f in FUEL, y in YEAR}:
	RateOfUseByTechnology[r,l,t,f,y] * YearSplit[l,y]
	=
	UseByTechnology[r,l,t,f,y];
```

In [ ]:
con = (m['RateOfUseByTechnology'] * ds['YearSplit']) - m['UseByTechnology'] == 0
mask = ds['InputActivityRatio'].sum('MODE_OF_OPERATION') != 0
m.add_constraints(con, name='Acc2_FuelUseByTechnology', mask=mask)

```ampl
s.t. Acc3_AverageAnnualRateOfActivity{r in REGION, t in TECHNOLOGY, m in MODE_OF_OPERATION, y in YEAR}:
	sum{l in TIMESLICE} RateOfActivity[r,l,t,m,y]*YearSplit[l,y]
	=
	TotalAnnualTechnologyActivityByMode[r,t,m,y];
```

In [ ]:
con = (m['RateOfActivity'] * ds['YearSplit']).sum('TIMESLICE') - m['TotalAnnualTechnologyActivityByMode'] == 0
mask = ds['OutputActivityRatio'].sum('FUEL') != 0
m.add_constraints(con, name='Acc3_AverageAnnualRateOfActivity', mask=mask)

```ampl
s.t. Acc4_ModelPeriodCostByRegion{r in REGION}:
	sum{y in YEAR}TotalDiscountedCost[r,y] = ModelPeriodCostByRegion[r];
```

In [ ]:
con = m['TotalDiscountedCost'].sum('YEAR') - m['ModelPeriodCostByRegion'] == 0
m.add_constraints(con, name='Acc4_ModelPeriodCostByRegion')

## Capital Costs

```ampl
s.t. CC1_UndiscountedCapitalInvestment{r in REGION, t in TECHNOLOGY, y in YEAR}: 
        CapitalCost[r,t,y] * NewCapacity[r,t,y] * CapitalRecoveryFactor[r,t] * PvAnnuity[r,t] 
        = 
        CapitalInvestment[r,t,y];
```

In [ ]:
con = ds['CapitalCost'] * m['NewCapacity'] * capital_recovery_factor * pv_annuity - m['CapitalInvestment'] == 0
mask = ~ds['CapitalCost'].isnull()
m.add_constraints(con, name='CC1_UndiscountedCapitalInvestment', mask=mask)

```ampl
s.t. CC2_DiscountingCapitalInvestment{r in REGION, t in TECHNOLOGY, y in YEAR}: 
    CapitalInvestment[r,t,y]  / DiscountFactor[r,y] = DiscountedCapitalInvestment[r,t,y];
```

In [ ]:
con = (m['CapitalInvestment'] / ds['DiscountRate']) - m['DiscountedCapitalInvestment'] == 0
mask = ~ds['CapitalCost'].isnull()
m.add_constraints(con, name='CC2_DiscountingCapitalInvestment', mask=mask)

## Salvage Value

### GNU MathProg implementation
```ampl
s.t. SV1_SalvageValueAtEndOfPeriod1{
    r in REGION, t in TECHNOLOGY, y in YEAR: 
        DepreciationMethod[r]=1 && 
        (y + OperationalLife[r,t]-1) > (max{yy in YEAR} max(yy)) && 
        DiscountRate[r]>0}: 
    SalvageValue[r,t,y] 
    = 
    CapitalCost[r,t,y] * NewCapacity[r,t,y] * CapitalRecoveryFactor[r,t] * PvAnnuity[r,t] * 
    (1-(((1+DiscountRate[r])^(max{yy in YEAR} max(yy) - y+1)-1)/((1+DiscountRate[r])^OperationalLife[r,t]-1)));
```
### Pyomo implementation
```python
def SalvageValueAtEndOfPeriod1_rule(model, r, t, y):
    if (
        model.DepreciationMethod[r] == 1
        and ((y + model.OperationalLife[r, t] - 1) > max(model.YEAR))
        and model.DiscountRate[r] > 0
    ):
        return model.SalvageValue[r, t, y] == model.CapitalCost[
            r, t, y
        ] * model.NewCapacity[r, t, y] * (
            1
            - (
                ((1 + model.DiscountRate[r]) ** (max(model.YEAR) - y + 1) - 1)
                / ((1 + model.DiscountRate[r]) ** model.OperationalLife[r, t] - 1)
            )
        )
    elif (
        model.DepreciationMethod[r] == 1
        and ((y + model.OperationalLife[r, t] - 1) > max(model.YEAR))
        and model.DiscountRate[r] == 0
    ) or (
        model.DepreciationMethod[r] == 2
        and (y + model.OperationalLife[r, t] - 1) > (max(model.YEAR))
    ):
        return model.SalvageValue[r, t, y] == model.CapitalCost[
            r, t, y
        ] * model.NewCapacity[r, t, y] * (
            1 - (max(model.YEAR) - y + 1) / model.OperationalLife[r, t]
        )
    else:
        return model.SalvageValue[r, t, y] == 0


model.SalvageValueAtEndOfPeriod1 = Constraint(
    model.REGION, model.TECHNOLOGY, model.YEAR, rule=SalvageValueAtEndOfPeriod1_rule
)
```

In [ ]:
def numerator(y: int):
    return ((1 + ds['DiscountRateIdv']) ** (max(ds.coords['YEAR']) - y + 1) - 1)

def denominator():
    return ((1 + ds['DiscountRateIdv']) ** ds['OperationalLife'] - 1)

def salvage_cost(ds):
    return ds['CapitalCost'].fillna(0) * (1 - (numerator(ds.coords['YEAR']) / denominator()))

con = m['SalvageValue'] - (m['NewCapacity'] * salvage_cost(ds)) == 0
mask = (ds['DepreciationMethod'] == 1) & ((ds.coords['YEAR'] + ds['OperationalLife'] - 1) > max(ds.coords['YEAR'])) & (ds['DiscountRateIdv'] > 0)
m.add_constraints(con, name='SV1_SalvageValueAtEndOfPeriod1', mask=mask)

```ampl
s.t. SV2_SalvageValueAtEndOfPeriod2{r in REGION, t in TECHNOLOGY, y in YEAR: 
        (DepreciationMethod[r]=1 && 
        (y + OperationalLife[r,t]-1) > (max{yy in YEAR} max(yy)) && 
        DiscountRate[r]=0) 
        || (DepreciationMethod[r]=2 && 
        (y + OperationalLife[r,t]-1) > (max{yy in YEAR} max(yy)))}: 
    SalvageValue[r,t,y] = CapitalCost[r,t,y] * NewCapacity[r,t,y] * CapitalRecoveryFactor[r,t] * PvAnnuity[r,t] *(1-(max{yy in YEAR} max(yy) - y+1)/OperationalLife[r,t]);
```

In [ ]:
def numerator(y: int):
    return 1 - (max(ds.coords['YEAR']) - y + 1) - 1

def denominator():
    return ds['OperationalLife']

def salvage_cost(ds):
    return ds['CapitalCost'].fillna(0) * (1 - (numerator(ds.coords['YEAR']) / denominator()))

con = m['SalvageValue'] - (m['NewCapacity'] * salvage_cost(ds)) == 0
mask = ((ds['DepreciationMethod'] == 1) & ((ds.coords['YEAR'] + ds['OperationalLife'] - 1) > max(ds.coords['YEAR'])) & (ds['DiscountRateIdv'] == 0)) | ((ds['DepreciationMethod'] == 2) & ((ds.coords['YEAR'] + ds['OperationalLife'] - 1) > max(ds.coords['YEAR'])))
m.add_constraints(con, name='SV2_SalvageValueAtEndOfPeriod2', mask=mask)

```ampl
s.t. SV3_SalvageValueAtEndOfPeriod3{r in REGION, t in TECHNOLOGY, y in YEAR: (y + OperationalLife[r,t]-1) <= (max{yy in YEAR} max(yy))}: 
    SalvageValue[r,t,y] = 0;
```

In [ ]:
con = m['SalvageValue'] == 0
mask = ((ds.coords['YEAR'] + ds['OperationalLife'] - 1) <= max(ds.coords['YEAR']))
m.add_constraints(con, name='SV3_SalvageValueAtEndOfPeriod3', mask=mask)

```ampl
s.t. SV4_SalvageValueDiscountedToStartYear{r in REGION, t in TECHNOLOGY, y in YEAR}: 
    DiscountedSalvageValue[r,t,y] = SalvageValue[r,t,y]/((1+DiscountRate[r])^(1+max{yy in YEAR} max(yy)-min{yy in YEAR} min(yy)));
```

In [ ]:
con = m['DiscountedSalvageValue'] - m['SalvageValue'] == 0
m.add_constraints(con, name='SV4_SalvageValueDiscountedToStartYear')

## Operating Costs

```ampl
s.t. OC1_OperatingCostsVariable{r in REGION, t in TECHNOLOGY, l in TIMESLICE, y in YEAR: sum{m in MODE_OF_OPERATION} VariableCost[r,t,m,y] <> 0}:
	sum{m in MODE_OF_OPERATION}
	TotalAnnualTechnologyActivityByMode[r,t,m,y] * VariableCost[r,t,m,y]
	=
	AnnualVariableOperatingCost[r,t,y];
```

In [ ]:
con = (m['TotalAnnualTechnologyActivityByMode'] * ds['VariableCost'].fillna(0)).sum(dims='MODE_OF_OPERATION') - m['AnnualVariableOperatingCost'] == 0
mask = (ds['VariableCost'].sum(dim='MODE_OF_OPERATION') != 0) & (~ds['VariableCost'].sum(dim='MODE_OF_OPERATION').isnull())
m.add_constraints(con, name='OC1_OperatingCostsVariable', mask=mask)

```ampl
s.t. OC2_OperatingCostsFixedAnnual{r in REGION, t in TECHNOLOGY, y in YEAR}:
	TotalCapacityAnnual[r,t,y]*FixedCost[r,t,y]
	=
	AnnualFixedOperatingCost[r,t,y];
```

In [ ]:
con = (m['TotalCapacityAnnual'] * ds['FixedCost']) - m['AnnualFixedOperatingCost'] == 0
mask = ~ds['FixedCost'].isnull()
m.add_constraints(con, name='OC2_OperatingCostsFixedAnnual', mask=mask)

```ampl
s.t. OC3_OperatingCostsTotalAnnual{r in REGION, t in TECHNOLOGY, y in YEAR}:
	AnnualFixedOperatingCost[r,t,y] + AnnualVariableOperatingCost[r,t,y]
	=
	OperatingCost[r,t,y];
```

In [ ]:
con = m['AnnualFixedOperatingCost'] + m['AnnualVariableOperatingCost'] - m['OperatingCost'] == 0
mask = (ds['VariableCost'].sum(dim='MODE_OF_OPERATION') != 0) & (~ds['FixedCost'].isnull())
m.add_constraints(con, name='OC3_OperatingCostsTotalAnnual', mask=mask)

```ampl
s.t. OC4_DiscountedOperatingCostsTotalAnnual{r in REGION, t in TECHNOLOGY, y in YEAR}:
	OperatingCost[r,t,y] / DiscountFactorMid[r, y]
	=
	DiscountedOperatingCost[r,t,y];
```

In [ ]:
con = m['OperatingCost'] / discount_factor_mid - m['DiscountedOperatingCost'] == 0
m.add_constraints(con, name='OC4_DiscountedOperatingCostsTotalAnnual')

## Total Discounted Costs

```ampl
s.t. TDC1_TotalDiscountedCostByTechnology{r in REGION, t in TECHNOLOGY, y in YEAR}: 
    DiscountedOperatingCost[r,t,y] + DiscountedCapitalInvestment[r,t,y] 
    + DiscountedTechnologyEmissionsPenalty[r,t,y] - DiscountedSalvageValue[r,t,y] = TotalDiscountedCostByTechnology[r,t,y];
```

In [ ]:
con = m['DiscountedOperatingCost'] + m['DiscountedCapitalInvestment'] + m['DiscountedTechnologyEmissionsPenalty'] \
        - m['DiscountedSalvageValue'] - m['TotalDiscountedCostByTechnology'] == 0
m.add_constraints(con, name='TDC1_TotalDiscountedCostByTechnology')

```ampl
s.t. TDC2_TotalDiscountedCost{r in REGION, y in YEAR}: 
    sum{t in TECHNOLOGY} TotalDiscountedCostByTechnology[r,t,y] + 
    sum{s in STORAGE} TotalDiscountedStorageCost[r,s,y] 
    = TotalDiscountedCost[r,y];
```

In [ ]:
con = m['TotalDiscountedCostByTechnology'].sum('TECHNOLOGY') + m['TotalDiscountedStorageCost'].sum('STORAGE') - m['TotalDiscountedCost'] == 0
m.add_constraints(con, name='TDC2_TotalDiscountedCost')

## Total Capacity Constraints

```ampl
s.t. TCC1_TotalAnnualMaxCapacityConstraint{r in REGION, t in TECHNOLOGY, y in YEAR: TotalAnnualMaxCapacity[r,t,y] <> -1}: 
    TotalCapacityAnnual[r,t,y] <= TotalAnnualMaxCapacity[r,t,y];
```

In [ ]:
con = m['TotalCapacityAnnual'] <= ds['TotalAnnualMaxCapacity']
mask = ds['TotalAnnualMaxCapacity'] >= 0
m.add_constraints(con, name='TCC1_TotalAnnualMaxCapacityConstraint', mask=mask)

```ampl
s.t. TCC2_TotalAnnualMinCapacityConstraint{r in REGION, t in TECHNOLOGY, y in YEAR: TotalAnnualMinCapacity[r,t,y]>0}: 
    TotalCapacityAnnual[r,t,y] >= TotalAnnualMinCapacity[r,t,y];
```

In [ ]:
con = m['TotalCapacityAnnual'] >= ds['TotalAnnualMinCapacity']
mask = ds['TotalAnnualMinCapacity'] > 0
m.add_constraints(con, name='TCC2_TotalAnnualMinCapacityConstraint', mask=mask)

# Annual Activity Constraints

```ampl
s.t. AAC1_TotalAnnualTechnologyActivity{r in REGION, t in TECHNOLOGY, y in YEAR}: 
    sum{l in TIMESLICE} RateOfTotalActivity[r,t,l,y]*YearSplit[l,y] = TotalTechnologyAnnualActivity[r,t,y];
```



In [ ]:
con = (m['RateOfTotalActivity'] * ds['YearSplit']).sum('TIMESLICE') - m['TotalTechnologyAnnualActivity'] == 0
m.add_constraints(con, name='AAC1_TotalAnnualTechnologyActivity')

```ampl
s.t. AAC2_TotalAnnualTechnologyActivityUpperLimit{r in REGION, t in TECHNOLOGY, y in YEAR: TotalTechnologyAnnualActivityUpperLimit[r,t,y] <> -1}:
    TotalTechnologyAnnualActivity[r,t,y] <= TotalTechnologyAnnualActivityUpperLimit[r,t,y] ;
```

In [ ]:
con = m['TotalTechnologyAnnualActivity'] <= ds['TotalTechnologyAnnualActivityUpperLimit']
mask = ds['TotalTechnologyAnnualActivityUpperLimit'] >= 0
m.add_constraints(con, name='AAC2_TotalAnnualTechnologyActivityUpperLimit', mask=mask)

```ampl
s.t. AAC3_TotalAnnualTechnologyActivityLowerLimit{r in REGION, t in TECHNOLOGY, y in YEAR: TotalTechnologyAnnualActivityLowerLimit[r,t,y]>0}: 
    TotalTechnologyAnnualActivity[r,t,y] >= TotalTechnologyAnnualActivityLowerLimit[r,t,y] ;
```

In [ ]:
con = m['TotalTechnologyAnnualActivity'] >= ds['TotalTechnologyAnnualActivityLowerLimit']
mask = ds['TotalTechnologyAnnualActivityLowerLimit'] > 0
m.add_constraints(con, name='AAC3_TotalAnnualTechnologyActivityLowerLimit', mask=mask)

# Total Activity Constraints

```ampl
s.t. TAC1_TotalModelHorizonTechnologyActivity{r in REGION, t in TECHNOLOGY}: 
    sum{y in YEAR} TotalTechnologyAnnualActivity[r,t,y] = TotalTechnologyModelPeriodActivity[r,t];
```

In [ ]:
con = m['TotalTechnologyAnnualActivity'].sum('YEAR') - m['TotalTechnologyModelPeriodActivity'] == 0
m.add_constraints(con, name='TAC1_TotalModelHorizonTechnologyActivity')

```ampl
s.t. TAC2_TotalModelHorizonTechnologyActivityUpperLimit{r in REGION, t in TECHNOLOGY: TotalTechnologyModelPeriodActivityUpperLimit[r,t]<>-1}: 
    TotalTechnologyModelPeriodActivity[r,t] <= TotalTechnologyModelPeriodActivityUpperLimit[r,t] ;
```

In [ ]:
con = m['TotalTechnologyModelPeriodActivity'] <= ds['TotalTechnologyModelPeriodActivityUpperLimit']
mask = ds['TotalTechnologyModelPeriodActivityUpperLimit'] >= 0
m.add_constraints(con, name='TAC2_TotalModelHorizonTechnologyActivityUpperLimit', mask=mask)

```ampl
s.t. TAC3_TotalModelHorizenTechnologyActivityLowerLimit{r in REGION, t in TECHNOLOGY: TotalTechnologyModelPeriodActivityLowerLimit[r,t]>0}: 
    TotalTechnologyModelPeriodActivity[r,t] >= TotalTechnologyModelPeriodActivityLowerLimit[r,t] ;
```

In [ ]:
con = m['TotalTechnologyModelPeriodActivity'] >= ds['TotalTechnologyModelPeriodActivityLowerLimit']
mask = ds['TotalTechnologyModelPeriodActivityLowerLimit'] > 0
m.add_constraints(con, name='TAC3_TotalModelHorizenTechnologyActivityLowerLimit', mask=mask)

## Emissions

```ampl
s.t. E1_AnnualEmissionProductionByMode{r in REGION, t in TECHNOLOGY, e in EMISSION, m in MODE_OF_OPERATION, y in YEAR:
									   EmissionActivityRatio[r,t,e,m,y] <> 0}:
	EmissionActivityRatio[r,t,e,m,y] * TotalAnnualTechnologyActivityByMode[r,t,m,y]
	=
	AnnualTechnologyEmissionByMode[r,t,e,m,y];
```


In [ ]:
if ~ds.coords['EMISSION'].isnull():
    mask = ~ds['EmissionActivityRatio'].isnull()
    con = ds['EmissionActivityRatio'] * m['TotalAnnualTechnologyActivityByMode'] - m['AnnualTechnologyEmissionByMode'] == 0
    m.add_constraints(con, name='E1_AnnualEmissionProductionByMode', mask=mask)

```ampl
s.t. E2_AnnualEmissionProduction{r in REGION, t in TECHNOLOGY, e in EMISSION, y in YEAR}:
	sum{m in MODE_OF_OPERATION}
	AnnualTechnologyEmissionByMode[r,t,e,m,y]
	=
	AnnualTechnologyEmission[r,t,e,y];
```

In [ ]:
if ~ds.coords['EMISSION'].isnull():
    con = m['AnnualTechnologyEmissionByMode'].sum(dims='MODE_OF_OPERATION') - m['AnnualTechnologyEmission'] == 0
    m.add_constraints(con, name='E2_AnnualEmissionProduction')

```ampl
s.t. E6_EmissionsAccounting1{r in REGION, e in EMISSION, y in YEAR}:
	sum{t in TECHNOLOGY}
	AnnualTechnologyEmission[r,t,e,y]
	=
	AnnualEmissions[r,e,y];
```

In [ ]:
if ~ds.coords['EMISSION'].isnull():
    con = m['AnnualTechnologyEmission'].sum(dims=['TECHNOLOGY']) - m['AnnualEmissions'] == 0
    m.add_constraints(con, name='E6_EmissionsAccounting1')

# Objective Function
```ampl
minimize cost: sum{r in REGION, y in YEAR} TotalDiscountedCost[r,y];
```

In [ ]:
objective = m['TotalDiscountedCost'].sum(dims=['REGION', 'YEAR'])
m.add_objective(expr=objective, overwrite=True)

# Solving

In [ ]:
m.to_file('simplicity.lp')

In [ ]:
print(available_solvers)

In [ ]:
m.solve(io_api='direct', log_fn='gurobi.log')

In [ ]:
m.variables.get_name_by_label(61)

In [ ]:
# m.constraints.get_name_by_label(43)

In [ ]:
m.solution

In [ ]:
df = m.solution[['AnnualEmissions']].to_dataframe()
df[df['AnnualEmissions'] != 0].plot()

In [ ]:
m.solution['OperatingCost'].to_dataframe()

In [ ]:
m.solution['DiscountedSalvageValue'].to_dataframe()

In [ ]:
m.solution['TotalDiscountedCost'].to_dataframe()

In [ ]:
from get_vars import parse_gmpl_code

with open('OSeMOSYS.txt', 'r') as textfile:
    osemosys = textfile.readlines()

    sets, params, vars = parse_gmpl_code("".join(osemosys))

In [ ]:
with open('auto_osemosys.py', 'w') as write_file:
    for name, variable in vars.items():
        lower = -inf
        upper = inf
        integer = False
        indices = [x.split(' in ')[-1].strip() for x in variable['indices']]
        if variable.get('bounds'):
            bounds = variable['bounds'].split('=')
            sign = bounds[0].strip()
            value = bounds[1].strip()
            if sign == '>':
                lower = value
            elif sign == '<':
                upper = value
        if variable.get('integer'):
            integer = True

        coords = ", ".join(["ds.coords['" + str(x) + "']" for x in indices])
        variable_name = ''.join('_'+c.lower() if c.isupper() else c for c in name).strip('_')
        write_file.write((f"coords = [{coords}]\n"))
        write_file.write(f"{variable_name} = m.add_variables(lower={lower}, upper={upper}, coords=coords, name='{name}', integer={integer})\n")
